In [1]:
import pandas as pd
features = pd.read_csv('/Users/johannessolibieda/Documents/GitHub/Its-Wekk/5 - Models/4 - LinReg/0 - ModelData/50MostImpFeatures_DF.csv')
target = pd.read_csv('/Users/johannessolibieda/Documents/GitHub/Its-Wekk/5 - Models/4 - LinReg/0 - ModelData/TargetCutto50MostImpFeatures_DF.csv')

In [2]:
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

import matplotlib.pyplot as plt
# Ensure the features and target dataframes are correctly assigned
features = features.drop(columns=['Datum'])
target = target['PM10_Combined_Trend_Residual']

# Split the data into train, test, and validate sets
train_data, temp_data, train_target, temp_target = train_test_split(features, target, test_size=0.3, random_state=42)
test_data, validate_data, test_target, validate_target = train_test_split(temp_data, temp_target, test_size=0.3333, random_state=42)

from sklearn.preprocessing import StandardScaler
import pickle

# Add squared features
features_squared = features ** 2
features = pd.concat([features, features_squared.add_suffix('_squared')], axis=1)

# Standardize the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Save the scaler using pickle
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Update the train, test, and validate sets with the scaled features
train_data, temp_data, train_target, temp_target = train_test_split(features_scaled, target, test_size=0.3, random_state=42)
test_data, validate_data, test_target, validate_target = train_test_split(temp_data, temp_target, test_size=0.3333, random_state=42)

# Define the parameter grid for GridSearchCV
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'eta0': [0.001, 0.01, 0.1, 1],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'penalty': ['l2', 'l1', 'elasticnet']
}

# Initialize the SGDRegressor
sgd = SGDRegressor(max_iter=1000, tol=1e-3)

# Initialize GridSearchCV
grid_search = GridSearchCV(sgd, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(train_data, train_target.values.ravel())

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Plot the progress bar
progress_bar = tqdm(total=100)
for i in range(100):
    progress_bar.update(1)
progress_bar.close()

# Print the best parameters and best score
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Evaluate the model on the test set
predictions = grid_search.predict(test_data)
mse = mean_squared_error(test_target, predictions)
print("Mean Squared Error on Test Set:", mse)

# Evaluate the model on the validation set
validate_predictions = grid_search.predict(validate_data)
validate_mse = mean_squared_error(validate_target, validate_predictions)
print("Mean Squared Error on Validation Set:", validate_mse)


# Expanding CV Fold Sizes
initial_train_size = int(0.5 * len(train_data))
test_size = int(0.1 * len(train_data))
num_folds = (len(train_data) - initial_train_size) // test_size

for fold in range(num_folds):
    start_train_size = initial_train_size + fold * test_size
    end_train_size = start_train_size + test_size

    X_train_fold = train_data[:end_train_size]
    y_train_fold = train_target[:end_train_size]
    X_test_fold = train_data[end_train_size:end_train_size + test_size]
    y_test_fold = train_target[end_train_size:end_train_size + test_size]

    model = SGDRegressor(**best_params, max_iter=1000, tol=1e-3)
    model.fit(X_train_fold, y_train_fold)
    fold_predictions = model.predict(X_test_fold)
    fold_mse = mean_squared_error(y_test_fold, fold_predictions)
    print(f"Fold {fold + 1}/{num_folds}, MSE: {fold_mse}")

Fitting 5 folds for each of 192 candidates, totalling 960 fits
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l2; total time=   0.1s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l2; total time=   0.1s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l1; total time=   0.1s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l2; total time=   0.1s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l1; total time=   0.1s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l2; total time=   0.1s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l2; total time=   0.1s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l1; total time=   0.1s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l1; total time=   0.1s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=elasticnet; total time=   0.1s
[CV] END alpha=0.0001, eta0=0

/Users/johannessolibieda/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV] END alpha=0.0001, eta0=1, learning_rate=constant, penalty=elasticnet; total time=   0.1s
[CV] END alpha=0.0001, eta0=1, learning_rate=constant, penalty=l1; total time=   0.1s
[CV] END alpha=0.0001, eta0=1, learning_rate=constant, penalty=elasticnet; total time=   0.1s
[CV] END alpha=0.0001, eta0=0.1, learning_rate=adaptive, penalty=l1; total time=   2.8s
[CV] END alpha=0.0001, eta0=1, learning_rate=constant, penalty=elasticnet; total time=   0.1s
[CV] END alpha=0.0001, eta0=1, learning_rate=constant, penalty=elasticnet; total time=   0.2s
[CV] END alpha=0.0001, eta0=1, learning_rate=constant, penalty=elasticnet; total time=   0.1s
[CV] END alpha=0.0001, eta0=0.1, learning_rate=adaptive, penalty=l1; total time=   3.0s
[CV] END alpha=0.0001, eta0=0.1, learning_rate=adaptive, penalty=elasticnet; total time=   3.3s
[CV] END alpha=0.0001, eta0=1, learning_rate=optimal, penalty=l2; total time=   0.6s
[CV] END alpha=0.0001, eta0=1, learning_rate=optimal, penalty=l2; total time=   0.7s
[C

/Users/johannessolibieda/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV] END alpha=0.01, eta0=0.001, learning_rate=optimal, penalty=l2; total time=   5.9s


...

[CV] END alpha=0.1, eta0=1, learning_rate=optimal, penalty=elasticnet; total time=  12.9s


/Users/johannessolibieda/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV] END alpha=0.1, eta0=1, learning_rate=optimal, penalty=elasticnet; total time=  12.9s
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, penalty=l1; total time=   3.3s
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, penalty=elasticnet; total time=   2.4s
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, penalty=l1; total time=   3.4s
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, penalty=l1; total time=   3.8s
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, penalty=elasticnet; total time=   1.9s
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, penalty=l1; total time=   3.7s
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, penalty=l1; total time=   3.6s
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, penalty=elasticnet; total time=15.0min
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, penalty=elasticnet; total time=15.0min
[CV] END alpha=0.1, eta0=1, learning_rate=adaptive, penalty=elasticnet; total time=15.0min


100%|██████████| 100/100 [00:00<00:00, 529583.84it/s]


Best Parameters: {'alpha': 0.01, 'eta0': 0.001, 'learning_rate': 'adaptive', 'penalty': 'elasticnet'}
Best Score: -36.85972845827638
Mean Squared Error on Test Set: 53.420429680559174
Mean Squared Error on Validation Set: 39.43080620653257
Fold 1/5, MSE: 41.002575000154366
Fold 2/5, MSE: 50.57108414873241
Fold 3/5, MSE: 32.74480928954214
Fold 4/5, MSE: 36.467927369417126
Fold 5/5, MSE: 12.820584057832388
